# Imports

In [ ]:
!pip install streamlit

In [ ]:
!pip install streamlit_lightweight_charts

In [ ]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz

!tar -xzf ta-lib-0.4.0-src.tar.gz
%cd ta-lib/
!./configure --prefix=$HOME
!make
!make install

!TA_LIBRARY_PATH=~/lib TA_INCLUDE_PATH=~/include pip install ta-lib

In [ ]:
!npm install localtunnel

In [ ]:
!pip install --force-reinstall --no-deps bokeh==2.4.3

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st
from bokeh.plotting import figure, column
import talib

# First Glance At Dataset

# EDA

## Dataset

In [49]:
%%writefile Dataset_Overview.py

import pandas as pd
import streamlit as st
from bokeh.plotting import figure, column
import talib
import datetime

# Setting Streamlit Page Config
st.set_page_config(layout='wide', page_title='Dataset Overview')

# Dataset
@st.cache_data
def load_dataset():
  df = pd.read_csv('/content/final.csv', index_col=0, parse_dates=['Date']).drop(columns=['time_stamp_x', 'time_stamp_y']).rename(columns={'Open': 'xmr_open',
                                                                                                                                    'Close': 'xmr_close',
                                                                                                                                    'High': 'xmr_high',
                                                                                                                                    'Low': 'xmr_low',
                                                                                                                                    'Adj Close': 'xmr_adj_close',
                                                                                                                                    'Volume': 'xmr_volume'})
  df['xmr_bar_color'] = df[['xmr_open', 'xmr_close']].apply(lambda o: 'red' if o.xmr_open > o.xmr_close else 'green', axis=1)
  df['date_str'] = df['Date'].astype(str)

  # Indicators
  df['xmr_sma'] = talib.SMA(df['xmr_close'], timeperiod=3)
  df['xmr_ma'] = talib.MA(df['xmr_close'], timeperiod=3)
  df['xmr_ema'] = talib.EMA(df['xmr_close'], timeperiod=3)
  df['xmr_rsi'] = talib.RSI(df['xmr_close'], timeperiod=3)
  df['xmr_mom'] = talib.MOM(df['xmr_close'], timeperiod=3)
  df['xmr_dema'] = talib.DEMA(df['xmr_close'], timeperiod=3)
  df['xmr_tema'] = talib.TEMA(df['xmr_close'], timeperiod=3)

  return df

# Dataset Overview
st.header("Dataset Overview")
df = load_dataset()


st.sidebar.subheader('Date Range Selection')
col1, col2 = st.sidebar.columns(2)
with col1:
  start_date = st.date_input(label='Start Date: ', value=datetime.date(2017, 11, 9))
with col2:
  end_date = st.date_input(label='End Date: ', value=datetime.date(2023, 10, 7))



sub_df = df.set_index('Date').loc[str(start_date):str(end_date)]
sub_df = sub_df.reset_index()

st.sidebar.subheader('Select Columns:')
columns = st.sidebar.multiselect(label='', options=sub_df.columns, default=list(sub_df.columns))

st.dataframe(sub_df[columns])


Overwriting Dataset_Overview.py


## XMR

In [50]:
!mkdir -p pages

In [51]:
%%writefile pages/XMR_Candlestick_Analysis.py



import pandas as pd
import streamlit as st
from bokeh.plotting import figure, column
import talib
import datetime

# Setting Streamlit Page Config
st.set_page_config(layout='wide', page_title="XMR Candlestick Analysis")

# Dataset
@st.cache_data
def load_dataset():
  df = pd.read_csv('/content/final.csv', index_col=0, parse_dates=['Date']).drop(columns=['time_stamp_x', 'time_stamp_y']).rename(columns={'Open': 'xmr_open',
                                                                                                                                    'Close': 'xmr_close',
                                                                                                                                    'High': 'xmr_high',
                                                                                                                                    'Low': 'xmr_low',
                                                                                                                                    'Adj Close': 'xmr_adj_close',
                                                                                                                                    'Volume': 'xmr_volume'})
  df['xmr_bar_color'] = df[['xmr_open', 'xmr_close']].apply(lambda o: 'red' if o.xmr_open > o.xmr_close else 'green', axis=1)
  df['date_str'] = df['Date'].astype(str)

  # Indicators
  df['xmr_sma'] = talib.SMA(df['xmr_close'], timeperiod=3)
  df['xmr_ma'] = talib.MA(df['xmr_close'], timeperiod=3)
  df['xmr_ema'] = talib.EMA(df['xmr_close'], timeperiod=3)
  df['xmr_rsi'] = talib.RSI(df['xmr_close'], timeperiod=3)
  df['xmr_mom'] = talib.MOM(df['xmr_close'], timeperiod=3)
  df['xmr_dema'] = talib.DEMA(df['xmr_close'], timeperiod=3)
  df['xmr_tema'] = talib.TEMA(df['xmr_close'], timeperiod=3)

  return df


df = load_dataset()


# Chart
indicator_colors = {
    'xmr_sma': 'orange',
    'xmr_ema': 'violet',
    'xmr_wma': 'blue',
    'xmr_rsi': 'yellow',
    'xmr_mom': 'black',
    'xmr_dema': 'red',
    'xmr_ma': 'tomato',
    'xmr_tema': 'dodgerblue'
}

def create_chart(df, close_line=False, include_vol=False, indicators={}):
  candle = figure(x_axis_type='datetime',
                  plot_height=500,
                  x_range=(df['Date'].values[0], df['Date'].values[-1]),
                  tooltips=[('Date', '@date_str'), ('xmr_open', '@xmr_open'), ('xmr_high', '@xmr_high'), ('xmr_low', '@xmr_low'), ('xmr_close', '@xmr_close')])

  candle.segment('Date', 'xmr_low', 'Date', 'xmr_high', color='black', line_width=0.5, source=df)
  candle.segment('Date', 'xmr_open', 'Date', 'xmr_close', color='xmr_bar_color', line_width=2 if len(df) > 100 else 6, source=df)

  candle.xaxis.axis_label = 'Date'
  candle.yaxis.axis_label = 'Price'

  if close_line:
    candle.line('Date', 'xmr_close', color='black', source=df)

  for indicator in indicators:
    candle.line('Date', indicator, color=indicator_colors[indicator], line_width=2, source=df, legend_label=indicator)

  # Volume Bars
  volume = None
  if include_vol:
    volume = figure(x_axis_type='datetime', plot_height=150, x_range=(df['Date'].values[0], df['Date'].values[-1]))
    volume.segment('Date', 0, 'Date', 'xmr_volume', line_width= 2 if len(df) > 100 else 6, line_color='xmr_bar_color', alpha=0.8, source=df)

  return column(children=[candle, volume], sizing_mode='stretch_width') if volume else candle

talib_indicators = ['xmr_ma', 'xmr_ema', 'xmr_sma', 'xmr_wma', 'xmr_rsi', 'xmr_mom', 'xmr_dema', 'xmr_tema']


# Dashboard
st.header('XMR :green[Candle]:red[stick] Analysis :tea::coffee:')

st.sidebar.subheader('Date Range Selection')
col1, col2 = st.sidebar.columns(2)
with col1:
  start_date = st.date_input(label='Start Date: ', value=datetime.date(2017, 11, 9))
with col2:
  end_date = st.date_input(label='End Date: ', value=datetime.date(2023, 10, 7))


close_line = st.sidebar.checkbox(label='Close Price Line')
volume = st.sidebar.checkbox(label='Include Volume')

indicators = st.sidebar.multiselect(label='Technical Indicators: ', options=talib_indicators)
st.sidebar.caption('Read More About Technial Indicators: https://blog.elearnmarkets.com/best-25-technical-indicators/')

sub_df = df.set_index('Date').loc[str(start_date):str(end_date)]
sub_df = sub_df.reset_index()

fig = create_chart(sub_df, close_line, volume, indicators)
st.bokeh_chart(fig, use_container_width=True)



Overwriting pages/XMR_Candlestick_Analysis.py


# Running the Code

In [52]:
!streamlit run Dataset_Overview.py &>/content/logs.txt & curl ipv4.icanhazip.com

34.125.74.7


In [ ]:
!npx localtunnel --port 8501